In [ ]:
#Import Statements Here
import numpy as np
import pandas as pd
from statistics import mean
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn as skl
from sklearn import metrics
from scipy import stats
rng = np.random.default_rng()
from sklearn import linear_model
from sklearn import metrics
import os
import subprocess

In [ ]:
def getFromBucket(name_of_file_in_bucket):
    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file from the bucket to the current working space
    os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

    print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe = pd.read_csv(name_of_file_in_bucket)
    return my_dataframe


In [ ]:
def storeInBucket(df, name_of_file_loc):
    # get the bucket name
    
    # This snippet assumes you run setup first

    # This code saves your dataframe into a csv file in a "data" folder in Google Bucket

    # Replace df with THE NAME OF YOUR DATAFRAME
    my_dataframe = df   

    # Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
    destination_filename = name_of_file_loc

    ########################################################################
    ##
    ################# DON'T CHANGE FROM HERE ###############################
    ##
    ########################################################################

    # save dataframe in a csv file in the same workspace as the notebook
    my_dataframe.to_csv(destination_filename, index=False)

    # get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')

    # copy csv file to the bucket
    args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
    output = subprocess.run(args, capture_output=True)

    # print output from gsutil
    output.stderr
    

In [ ]:
AvgHeartRateDf = getFromBucket("avg_heart_rate_median.csv")
MedHeartRateDf = getFromBucket("heart_rate_min_max_median.csv")
SleepSummaryDf = getFromBucket("sleep_summary_stat_df.csv")
MedianStepsDf = getFromBucket("steps_median.csv")
WearTimeDf = getFromBucket("fitbit_avg_weartime.csv")

In [ ]:
#Make a function to get all the covid-negative data from the bucket as well. We are going to compare the two

In [ ]:
NegMedianStepsDf= getFromBucket('negative_cohort_steps_median.csv')
NegSleepSummaryDf= getFromBucket('negative_cohort_sleep_summary_stat_df.csv')
NegMedHeartRateDf = getFromBucket('negative_cohort_heart_rate_min_max_median.csv')
NegWearTimeDf= getFromBucket('negative_cohort_fitbit_avg_weartime.csv')
NegAvgHeartRateDf= getFromBucket('negative_cohort_avg_heart_rate_median.csv')

In [ ]:
#Make these below an iterable function, and use that function to include positive and negative data sets

def variableClean(df, beforeVar, afterVar, beforeVarMax, afterVarMax):
    
    returnDf = df.sort_values('person_id', ignore_index=True)
    returnDf  = returnDf[returnDf[beforeVar]>0]
    returnDf  = returnDf[returnDf[afterVar]>0]
    
    #remove outliers
    returnDf  = returnDf[returnDf[beforeVar]<beforeVarMax]
    returnDf  = returnDf[returnDf[afterVar]<afterVarMax]
    
    return returnDf

In [ ]:
AvgHeartRateDf= variableClean(AvgHeartRateDf, 'before_covid_avg_heart_rate', 'after_covid_avg_heart_rate', 100, 100)
MedHeartRateDf= variableClean(MedHeartRateDf, 'before_covid_min_heart_rate', 'after_covid_min_heart_rate', 125, 140)
SleepSummaryDf = variableClean(SleepSummaryDf, 'before_covid_median_minutes_asleep', 'after_covid_median_minutes_asleep', 600, 1000)
MedianStepsDf= variableClean(MedianStepsDf, 'before_covid_median_steps', 'after_covid_median_steps', 100000, 100000)

In [ ]:
#clean the Negative Variables
NegAvgHeartRateDf= variableClean(NegAvgHeartRateDf, 'before_neg_covid_avg_heart_rate', 'after_neg_covid_avg_heart_rate', 100, 100)
NegMedHeartRateDf= variableClean(NegMedHeartRateDf, 'before_neg_covid_min_heart_rate', 'after_neg_covid_min_heart_rate', 125, 140)
NegSleepSummaryDf = variableClean(NegSleepSummaryDf, 'before_neg_covid_median_minutes_asleep', 'after_neg_covid_median_minutes_asleep', 600, 1000)
NegMedianStepsDf= variableClean(NegMedianStepsDf, 'before_neg_covid_test_median_steps', 'after_negative_covid_test_median_steps', 100000, 100000)

NegMedianStepsDf = NegMedianStepsDf.rename(columns = {'person_id': 'person_id', 'neg_covid_dt': 'neg_covid_dt', 
                                                      'before_neg_covid_test_median_steps': 'before_neg_covid_median_steps',
                                                      'after_negative_covid_test_median_steps': 'after_neg_covid_median_steps'})

In [ ]:
WearTimeDf=WearTimeDf.sort_values('person_id',ignore_index=True)
len(WearTimeDf[WearTimeDf['average_weartime']<8])
#WearTimeDf

In [ ]:
WearTimeDf['average_weartime'].plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
plt.title('Average Wear time per day')
plt.ylabel('Counts')
plt.xlabel('Average Hours Worn Per Day')
plt.grid(axis='y', alpha=0.75)

In [ ]:
FitBitList = [AvgHeartRateDf, MedHeartRateDf, SleepSummaryDf, MedianStepsDf]
for a in FitBitList:
    print(len(a))

In [ ]:
#Add in Sleep Quality Data
#SleepLevelDf = pd.read_csv('exploring_sleep_level_per_minute_indicator.csv')
#MinutesDeepSleepDf = pd.read_csv('exploring_deep_sleep_mins.csv')
#SleepLevelDf 
#MinutesDeepSleepDf

## Moble Device Visualization

In [ ]:
#Make a function to give us a box plot using before and after variables
def makeBoxPlots(varString, df, neg=False):
    
    beforeVar = 'before_covid_' + varString
    afterVar = 'after_covid_' + varString
    
    if neg:
        beforeVar = 'before_neg_covid_' + varString
        afterVar = 'after_neg_covid_' + varString
        
    before = df[beforeVar]
    after = df[afterVar]

    fig, ax = plt.subplots()
    ax.boxplot([before,after])

    ax.set_title(varString +' Before and After Covid')
    ax.set_xlabel('')
    ax.set_ylabel(varString)
    xticklabels=['Before Covid' , 'After Covid']
    ax.set_xticklabels(xticklabels)

    ax.yaxis.grid(True)

    plt.show()

    

In [ ]:
#Make a function to give us a histogram in the change of those before and after variables
def makeDifferenceHist(varString, df,neg=False):
    
    beforeVar = 'before_covid_' + varString
    afterVar = 'after_covid_' + varString
    
    if neg:
        beforeVar = 'before_neg_covid_' + varString
        afterVar = 'after_neg_covid_' + varString
        
    before = df[beforeVar]
    after = df[afterVar]
    
    diff = after - before
    print(mean(diff))

    diff.plot.hist(grid=True, bins=20, rwidth=0.9,
                   color='#607c8e')
    plt.title('Change in '+ varString+ ' After Covid')
    plt.ylabel('Counts')
    plt.xlabel('Change in '+ varString)
    plt.grid(axis='y', alpha=0.75)
    
    print(stats.ttest_rel(before, after))

    

In [ ]:
makeBoxPlots('avg_heart_rate', AvgHeartRateDf)

In [ ]:
makeDifferenceHist('avg_heart_rate', AvgHeartRateDf)

In [ ]:
makeBoxPlots('min_heart_rate', MedHeartRateDf)

In [ ]:
makeDifferenceHist('min_heart_rate', MedHeartRateDf)

In [ ]:
makeBoxPlots('max_heart_rate', MedHeartRateDf)

In [ ]:
makeDifferenceHist('max_heart_rate', MedHeartRateDf)

In [ ]:
makeBoxPlots('median_minutes_asleep', SleepSummaryDf)

In [ ]:
makeDifferenceHist('median_minutes_asleep', SleepSummaryDf)

In [ ]:
makeBoxPlots('median_steps', MedianStepsDf)

In [ ]:
makeDifferenceHist('median_steps', MedianStepsDf)

In [ ]:
Dflist = [AvgHeartRateDf, MedHeartRateDf, SleepSummaryDf, MedianStepsDf]


## Compare Negative cases as well

In [ ]:
#NegMedianStepsDf

In [ ]:
makeBoxPlots('median_steps', NegMedianStepsDf, neg=True)

In [ ]:
makeDifferenceHist('median_steps', NegMedianStepsDf, neg=True)

In [ ]:
def statTestPosNegDifference(posDf, negDf, posVar, negVar):
    
    posBeforeStr = 'before_covid_' + posVar
    posAfterStr = 'after_covid_' +posVar
    posDiff = posDf[posAfterStr] - posDf[posBeforeStr]
    
    negBeforeStr = 'before_neg_covid_' + negVar
    negAfterStr = 'after_neg_covid_' +negVar
    negDiff = negDf[negAfterStr] - negDf[negBeforeStr]
    
    covidDiff = posDiff - negDiff
    
    a = stats.ttest_ind(negDiff, posDiff)
    
    return a

In [ ]:
statTestPosNegDifference(MedianStepsDf, NegMedianStepsDf, 'median_steps', 'median_steps')

In [ ]:
statTestPosNegDifference(SleepSummaryDf, NegSleepSummaryDf, 'median_minutes_asleep', 'median_minutes_asleep')

In [ ]:
statTestPosNegDifference(MedHeartRateDf, NegMedHeartRateDf, 'max_heart_rate', 'max_heart_rate')

In [ ]:
statTestPosNegDifference(MedHeartRateDf, NegMedHeartRateDf, 'min_heart_rate', 'min_heart_rate')

In [ ]:
statTestPosNegDifference(AvgHeartRateDf, NegAvgHeartRateDf, 'avg_heart_rate', 'avg_heart_rate')

## Export data for machine learning

In [ ]:
#find counts for who has all of the data combined, as well as who has each data type

#Define a function to merge all dataframes. Use inner or outer as arguements to the merge function

def joinAllDf(DataFrameList, mergeType, covidValue):
    #define merge based on CovidValue
    if covidValue =='pos':
        joinCol = 'min_covid_dt'
    if covidValue == 'neg':
        joinCol = 'neg_covid_dt'
    
    mergeDf = DataFrameList[0]
    
    for df in DataFrameList:
        mergeDf = pd.merge(left = mergeDf, right=df, how=mergeType, left_on=['person_id', joinCol] , right_on = ['person_id', joinCol])
    
    return mergeDf

In [ ]:
innerJoinPosDf = joinAllDf([AvgHeartRateDf, MedHeartRateDf, SleepSummaryDf, MedianStepsDf], 'inner', 'pos')
#innerJoinDf

In [ ]:
outerJoinPosDf = joinAllDf([AvgHeartRateDf, MedHeartRateDf, SleepSummaryDf, MedianStepsDf], 'outer', 'pos')
#outerJoinDf

In [ ]:
innerJoinNegDf = joinAllDf([NegAvgHeartRateDf, NegMedHeartRateDf, NegSleepSummaryDf, NegMedianStepsDf], 'inner', 'neg')
#innerJoinNegDf

In [ ]:
outerJoinNegDf = joinAllDf([NegAvgHeartRateDf, NegMedHeartRateDf, NegSleepSummaryDf, NegMedianStepsDf], 'outer', 'neg')
#outerJoinNegDf

In [ ]:
#bring the outer join Dfs together, and then do the same to the inner join

def combinePosAndNeg(NegDf, PosDf):
    
    NegDf['covid_test'] = NegDf.apply(lambda row: 0, axis=1)
    PosDf['covid_test'] = NegDf.apply(lambda row: 1, axis=1)
    
    #rename NegDf columns to match PosDf columns
    NegDf.columns = list(PosDf.columns)
    
    #concatinate the two so that they are in the sane space now that they have the same columns
    returnDf = pd.concat([PosDf, NegDf], axis=0)
    return returnDf
    

In [ ]:
OuterJoinMarch16 = combinePosAndNeg(outerJoinNegDf, outerJoinPosDf)
#storeInBucket(OuterJoinMarch16 , 'OuterJoinMarch16MobileDevice.csv')
OuterJoinMarch16

In [ ]:
InnerJoinMarch16 = combinePosAndNeg(innerJoinNegDf, innerJoinPosDf)
#storeInBucket(InnerJoinMarch16 , 'InnerJoinMarch16MobileDevice.csv')
InnerJoinMarch16 